In [1]:
import pandas as pd
from datetime import datetime

Import Data- Data must be uploaded as in order for it to be imported first

In [2]:
df = pd.read_csv("aita_clean.csv")

Helper functions to sort the data and identify characteristics for use as facotrs in the naive bayes net

In [3]:
def findWord(text, word):
  if word in text:
    return 1
  else:
    return 0

def authorGender(text):
  start = None
  end = None
  start = text.find("(")
  end = text.find(")")
  gender = None
  if end != None and start != None and end > start:
    gender = text[end-1]
  if gender == 'F' or gender == 'f':
    return "F"
  if gender == 'M' or gender == 'm':
    return "M"
    gender = text[start+1]
  if gender == 'F' or gender == 'f':
    return "F"
  if gender == 'M' or gender == 'm':
    return "M"
  else:
    return None

  
def authorGender2(text):
  if findWord(text, "female"):
    return "F"
  if findWord(text, "male"):
    return "M"
  return None



def profanity(text):
  if findWord(text, "fuck") or findWord(text, "bitch") or findWord(text, "shit") or findWord(text, "damn") or findWord(text, "cunt"):
    return 1
  else:
    return 0

def getHour(timestamp):
  return datetime.fromtimestamp(timestamp).hour

def weed(text):
  if findWord(text, "weed") or findWord(text, "marijuana"):
    return 1
  else:
    return 0

def drugs(text):
  if findWord(text, "crack") or findWord(text, "cocaine") or findWord(text, "meth") or findWord(text, "drug"):
    return 1
  else:
    return 0

def parents(text):
  if findWord(text, "mom") or findWord(text, "dad") or findWord(text, "mother") or findWord(text, "father") or findWord(text, "parents"):
    return 1
  else:
    return 0

def hypothetical(title, text):
  if findWord(title, "wibta") or findWord(text, "wibta"):
    return 1
  else:
    return 0

def gf(text):
  if findWord(text, "girlfriend") or findWord(text, "gf") or findWord(text, "wife"):
    return 1
  return 0

def bf(text):
  if findWord(text, "boyfriend") or findWord(text, "bf") or findWord(text, "husband"):
    return 1
  return 0

def kids(text):
  if findWord(text, "kids") or findWord(text, "son") or findWord(text, "daughter"):
    return 1
  return 0

def son(text):
  if findWord(text, "son"):
    return 1
  return 0

def daughter(text):
  if findWord(text, "daughter"):
    return 1
  return 0

Iterate through the data row by row, using the helper functions to identify the characteristics of the data

In [4]:
titleChars = []
textChars = []
isAsshole = []
authorgender = []
Parents = []
prof = []
Weed = []
Drugs = []
hypo = []
boyfriend = []
girlfriend = []
sons = []
daughters = []
children = []
hour = []
for row in df.iterrows():
  title = row[1]['title']
  text = row[1]['body']
  time = row[1]['timestamp']
  hr = getHour(time)
  titleLength = 0
  textLength = 0
  isA = "Asshole"
  if (type(title) == str):
    titleLength = len(title)
  if (type(text) == str):
    textLength = len(text)
    gender = authorGender(text.lower())
    if gender == None:
      gender = authorGender2(text.lower())
    mentionedParents = parents(text.lower())

    p = profanity(text.lower())
    w = weed(text.lower())
    d = drugs(text.lower())
    h = hypothetical(title.lower(), text.lower())
    b = bf(text.lower())
    g = gf(text.lower())
    child = kids(text.lower())
    s = son(text.lower())
    da = daughter(text.lower())
  if (row[1]['is_asshole'] == 0):
    isA = "Not Asshole"
  
  isAsshole.append(isA)
  titleChars.append(titleLength)
  textChars.append(textLength)
  authorgender.append(gender)
  Parents.append(mentionedParents)
  prof.append(p)
  Weed.append(w)
  Drugs.append(d)
  hypo.append(h)
  boyfriend.append(b)
  girlfriend.append(g)
  children.append(child)
  sons.append(s)
  daughters.append(da)
  hour.append(hr)


Append new characteristics to the data

In [5]:
df['Hour Posted'] = hour
df['Title Length'] = titleChars
df['Text Length'] = textChars
df['IsAsshole'] = isAsshole
df['Author Gender'] = authorgender
df['Mentions Parents'] = Parents
df['Mentions Weed'] = Weed
df['Mentions Drugs'] = Drugs
df['Profanity'] = prof
df['Hyopthetical'] = hypo
df['Mentions Girlfriend/Wife'] = girlfriend
df['Mentions Boyfriend/Husband'] = boyfriend
df['Mentions Kids'] = children
df['Mentions Son'] = sons
df['Mentions Daughter'] = daughters


Round the title length and text length to nearest 10, also group outliers together

In [6]:
for idx, row in df.iterrows():
  titleLength = row['Title Length']
  if titleLength < 20:
    df.at[idx, 'Title Length'] = 20
  elif titleLength > 100:
    df.at[idx, 'Title Length'] = 120
  else:
    df.at[idx, 'Title Length'] = round(titleLength/10)*10 

  textLength = row['Text Length']
  if textLength < 100:
    df.at[idx, 'Text Length'] = 100
  elif textLength > 3200:
    df.at[idx, 'Text Length'] = 3200
  else:
    df.at[idx, 'Text Length'] = round(textLength/10)*10 

Format the data for use in naive bayes, picking out the important characteristics instead of all of them. Also splits the data into a test and train section

In [7]:
nbData = pd.DataFrame()
nbData['IsAsshole'] = df['IsAsshole']
nbData['Title Length'] = df['Title Length']
nbData['Text Length'] = df['Text Length']
nbData['Hour'] = df['Hour Posted']
nbData['Parents'] = df['Mentions Parents']
nbData['Weed'] = df['Mentions Weed']
nbData['Drugs'] = df['Mentions Drugs']
nbData['Profanity'] = df['Profanity']
nbData['Hyopthetical'] = df['Hyopthetical']
nbData['Sex'] = df['Author Gender']
nbData['GF'] = df['Mentions Girlfriend/Wife']
nbData['BF'] = df['Mentions Boyfriend/Husband']
nbData['Kids'] = df['Mentions Kids']
print(nbData)
nbTest = nbData.iloc[90000:97627]
nbData = nbData.iloc[0:89999]

         IsAsshole  Title Length  Text Length  Hour  Parents  Weed  Drugs  \
0          Asshole            70          510    21        0     0      1   
1          Asshole            40          690    22        1     0      0   
2      Not Asshole            40          100    23        0     0      0   
3          Asshole            60          200     0        1     0      0   
4      Not Asshole           120          100     0        1     0      0   
...            ...           ...          ...   ...      ...   ...    ...   
97623  Not Asshole            60          740    17        1     0      0   
97624  Not Asshole           120         2310    17        1     0      0   
97625  Not Asshole            60         2030    17        1     0      0   
97626  Not Asshole            50         1200    17        1     0      1   
97627  Not Asshole            80         2000    17        0     0      0   

       Profanity  Hyopthetical   Sex  GF  BF  Kids  
0              0      

Get the prior probabilities

In [8]:
priors = nbData.groupby("IsAsshole").size().div(len(nbData))
print(priors)

IsAsshole
Asshole        0.273147
Not Asshole    0.726853
dtype: float64


Get the likelihoods

In [9]:
likelihoods = {}
for (columnName, columnData) in nbData.iteritems():
  likelihoods[columnName] = nbData.groupby(['IsAsshole', columnName]).size().div(len(nbData)).div(priors)
#print(likelihoods)
print(likelihoods['Sex']['Not Asshole']['F'])
print(likelihoods['Sex']['Not Asshole']['M'])

0.07594472300354653
0.06411275528922587


Iterate through the train data row by row to calculate the probability of asshole verdict vs not ashsole verdict. The larger probability is returned as the verdict and the total number of correct verdicts is compared against the size of the test data to get the accuracy

In [10]:
numCorrect = 0
numIncorrect = 0
for entry in nbTest.iterrows():
  row = entry[1]
  p_asshole = 1
  p_not_asshole = 1
  #print(entry)
  for (columnName, columnData) in nbData.iteritems():
    #print(columnName)
    #print(row[columnName])
    if columnName != "IsAsshole" and row[columnName] != None:
      p_asshole = p_asshole * likelihoods[columnName]['Asshole'][row[columnName]]
      p_not_asshole = p_not_asshole * likelihoods[columnName]['Not Asshole'][row[columnName]]
  p_asshole = p_asshole * priors['Asshole']
  p_not_asshole = p_not_asshole * priors['Not Asshole']
  if p_asshole > p_not_asshole:
    prediction = "Asshole"
  else:
    prediction = "Not Asshole"
  if prediction == row['IsAsshole']:
    numCorrect += 1
  else:
    numIncorrect += 1

accuracy = numCorrect/(numIncorrect + numCorrect)
print(accuracy)

0.7470827323980596
